In [2]:
import os
import pandas as pd
import numpy as np
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential
import tqdm
import glob

In [ ]:
# example on how cut-data looks
cut_data_dir = '/content/drive/MyDrive/Data/cut_data'
cut_csv_path = glob.glob(cut_data_dir + '/*.csv')


print(len(cut_csv_path))
print(cut_csv_path)

q = pd.read_csv(cut_csv_path[0])
print('shape:', q.shape)
q.head(100)


451
['/content/drive/MyDrive/Data/cut_data/d_spy_COL.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_F.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_GS.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_ETR.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_EBAY.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_COO.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_ECL.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_GLW.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_FDX.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_EXC.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_EXR.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_FAST.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_FE.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_GD.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_HES.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_DWDP.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_EL.csv', '/content/drive/MyDrive/Data/cut_data/d_spy_DRE.csv', '/content/drive/MyDrive/Da

,Unnamed: 0,Date,price,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound
0,0,2017-12-07,132.298141,-0.880858,-0.496778,0.000000,0.000000
1,1,2017-12-08,133.288330,0.000000,0.000000,-0.984410,0.000000
2,2,2017-12-11,133.199203,0.000000,0.000000,0.000000,0.000000
3,3,2017-12-12,133.199203,0.000000,0.000000,0.000000,0.000000
4,4,2017-12-13,132.892258,-0.785488,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
95,95,2018-04-26,131.981247,-0.728050,-0.677689,-0.585291,-0.692253
96,96,2018-04-27,132.020935,-0.739730,-0.638403,-0.562129,-0.674928
97,97,2018-04-30,131.564316,-0.805050,-0.663419,-0.714391,-0.703474
98,98,2018-05-01,131.256607,-0.698149,-0.715351,-0.828814,-0.713365


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#print(len(csv_file_path)) : 451

split = 0.85
sequence_length = 10
batch_size = 100
input_dim = 1
input_timesteps = 9
neurons = 50
epochs = 5
prediction_len = 1
dense_output = 1
drop_out = 0.2

mu = 0
noise = 0.1


def normalise_data(data_train, sequence_length):  # data_train is numpy array of shape(102,5)
    record_min = []
    record_max = []
    data_windows = []
    for i in range(len(data_train) - sequence_length):
        data_windows.append(data_train[i:i + sequence_length]) # sequence data at every 10 day timestamp
    data_windows = np.array(data_windows).astype(float)
    y_test_ori = data_windows[:, -1, [0]] # only useful for y test

    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]

    normalised_data = []

    for win_i in range(win_num):
        normalised_window = []
        for col_i in range(col_num):
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0: # price
                record_min.append(temp_min)
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)

    normalised_data = np.array(normalised_data)

    #By using normalised_data[:, :-1], the code is effectively extracting the input data for the LSTM model.
    #  Specifically, it is extracting all features from the first to the penultimate time step for each data sample, resulting in a new array of shape [92, 9, 5].
    #  This is because the input data for each sample has a length of sequence_length - 1 (i.e., 9 in this case) along the second dimension.

    x_train = normalised_data[:, :-1] #---> prices till 9 time stamps

    #y_train = normalised_data[:, -1, [0]]: This line extracts the last column of normalised_data and reshapes it to retain the 2-dimensional structure.
    #  The resulting y_train array contains the target variable that corresponds to the last time step of each input sequence in x_train.
    # In other words, it represents the next data point that the model needs to predict.
    y_train = normalised_data[:, -1, [0]] #---> price to predict after 9th time stamp

    return x_train, y_train, y_test_ori, record_min, record_max, window_data, normalised_data


def stock_lstm(list_path): # list_path = cut_csv_path
    df = pd.read_csv(list_path)
    #print("Before:", df)
    df_price = df.copy()

    split_index = int(len(df_price) * split)
    cols = ['price']
    data_train = df_price.get(cols).values[ : split_index]
    len_train = len(data_train)
    print('train', data_train.shape)
    data_test = df_price.get(cols).values[ split_index: ]
    len_test = len(data_test)
    print('test',data_test.shape)



    x_train , y_train , y_train_ori , train_record_min, train_record_max, train_window_data , train_normalised_data = normalise_data(data_train , sequence_length)

    x_test , y_test , y_test_ori , test_record_min, test_record_max, test_window_data , test_normalised_data = normalise_data(data_test , sequence_length)

    print('train shape',x_train.shape)
    print('train label shape',y_train.shape)
    #print(y_train_ori)
    print('test shape', x_test.shape)
    print('test lable shape', y_test.shape)


    # LSTM MODEL parameters
    # sequence_length = 10
    # batch_size = 100
    #input_dim = 5
    #input_timesteps = 10
    #neurons = 50
    #epochs = 5
    #prediction_len = 1
    #dense_output = 1
    #drop_out = 0.2
    model = Sequential()
    model.add(LSTM(neurons, input_shape = (input_timesteps, input_dim), return_sequences = True))
    model.add(Dropout(drop_out))
    model.add(LSTM(neurons, return_sequences = True))
    model.add(LSTM(neurons, return_sequences = False))
    model.add(Dropout(drop_out))
    model.add(Dense(dense_output, activation='linear'))

    # Compile Model
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')

    # Fit Model
    model.fit(x_train, y_train, batch_size, epochs )

    # Prediction
    data = x_test
    prediction_sequence = []
    pre_win_num = int(len(data)/prediction_len)
    window_size = sequence_length

    for i in range(0, pre_win_num):
        current_frame = data[i * prediction_len]
        predicted = []
        for j in range(0, prediction_len):
            # By using np.newaxis, we effectively increase the dimensionality of curr_frame from a 2D array
            # (shape: (input_timesteps, input_dim)) to a 3D array (shape: (1, input_timesteps, input_dim)).
            # This is done to match the expected shape for prediction with the LSTM model.
            temp = model.predict(current_frame[newaxis, :, :])[0]
            # the output of model.predict() is a 2D array with shape (1, dense_output)
            #  The [0] indexing is used to extract the predicted value from the 2D array and convert it to a scalar.
            predicted.append(temp)
            # after predicting the value at the current time step,
            # we need to shift the input sequence by one step to include the newly predicted value and exclude the oldest value from the sequence.
            current_frame = current_frame[1:]
            # Sliding window --> [ window_size - 2 ]
            # new_arr = np.insert(arr, index, row_to_insert, axis=0)
            # new_arr = np.insert(arr, index, column_to_insert, axis=1)
            current_frame = np.insert(current_frame, [window_size - 2], predicted[-1], axis = 0 )
        prediction_sequence.append(predicted)

    # denormalized predictions
    de_predict = []
    len_pre_win = int(len(data)/prediction_len)
    len_pre = prediction_len
    m = 0
    for i in range(0,len_pre_win):
        for j in range(0, len_pre):
            # prediction_seqs[i][j][0] retrieves the normalized prediction for the j-th prediction in the i-th window.
            # This prediction is denormalized back to the original scale using record_max[m] and record_min[m].
            # m is used to keep track of the corresponding maximum and minimum values used for normalization during the data preparation step.
            de_predict.append(prediction_sequence[i][j][0] * test_record_max[m] + test_record_min[m])
            # m is incremented after each prediction to keep track of the current index in record_max and record_min.
            m += 1
    #print('denormalized predictions', de_predict)

    # Error
    error = []
    squared_error = []
    absolute_error = []
    error_percent = []
    daily_accuracy = []
    diff = y_test.shape[0] - prediction_len * pre_win_num
    for i in range(y_test_ori.shape[0] - diff):
        error.append(y_test_ori[i,] - de_predict[i])

        # Calculate squared and absolute error for each data point
        squared_error.append(error[-1] * error[-1])
        absolute_error.append(abs(error[-1]))

        # Calculate and append accuracy for each day
        val = absolute_error[-1] / y_test_ori[i,]
        val = abs(val)
        error_percent.append(val)
        daily_mean_error_percent = sum(error_percent) / len(error_percent)
        daily_accuracy.append(1 - daily_mean_error_percent)

    # Calculate overall accuracy and MSE
    overall_accuracy = sum(daily_accuracy) / len(daily_accuracy)
    overall_MSE = sum(squared_error) / len(squared_error)

    return overall_MSE, overall_accuracy, daily_accuracy, y_test_ori, de_predict, x_train, y_train, x_test, y_test




result_LSTM_df_1 = pd.DataFrame(columns=('index', 'stock_name', 'MSE','Daily_Accuracy' ,'Overall_Accuracy', 'True_Price', 'Predicted_Price'))
m = len(cut_csv_path)
#filename = file_list




for i in tqdm.tqdm(range(0,m)):
    index = i
    stock = os.path.basename(cut_csv_path[i])[6:-4]
    print('stock name',stock)
    result = stock_lstm(cut_csv_path[i])
    MSE = result[0]
    accuracy = result[1]
    daily_accuracy = result[2]
    true_price = result[3]
    print('true_price', true_price)
    predicted = result[4]
    result_LSTM_df_1 = result_LSTM_df_1.append(pd.DataFrame({'index': [index],
                                                         'stock_name': [stock],
                                                         'MSE': [MSE],
                                                        'Daily_Accuracy':[daily_accuracy],
                                                         'Overall_Accuracy': [accuracy],
                                                        'True_Price': [true_price],
                                                        'Predicted_Price': [predicted]}), ignore_index = True)



  0%|          | 0/451 [00:00<?, ?it/s]

stock name COL
train (102, 1)
test (19, 1)
train shape (92, 9, 1)
train label shape (92, 1)
test shape (9, 9, 1)
test lable shape (9, 1)
Epoch 1/5
1/1 [==============================] - 11s 11s/step - loss: 0.3690
Epoch 2/5
1/1 [==============================] - 0s 14ms/step - loss: 0.3423
Epoch 3/5
1/1 [==============================] - 0s 21ms/step - loss: 0.3197
Epoch 4/5
1/1 [==============================] - 0s 20ms/step - loss: 0.2969
Epoch 5/5
1/1 [==============================] - 0s 22ms/step


<ipython-input-5-57c49f069867>:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_LSTM_df_1 = result_LSTM_df_1.append(pd.DataFrame({'index': [index],
  0%|          | 1/451 [00:20<2:34:02, 20.54s/it]

true_price [[135.669724]
 [136.804108]
 [137.570328]
 [137.182251]
 [137.132492]
 [137.421082]
 [137.013092]
 [137.291718]
 [136.833969]]
stock name F
train (102, 1)
test (19, 1)
train shape (92, 9, 1)
train label shape (92, 1)
test shape (9, 9, 1)
test lable shape (9, 1)
Epoch 1/5
1/1 [==============================] - 7s 7s/step - loss: 0.4199
Epoch 2/5
1/1 [==============================] - 0s 26ms/step - loss: 0.3916
Epoch 3/5
1/1 [==============================] - 0s 19ms/step - loss: 0.3685
Epoch 4/5
1/1 [==============================] - 0s 15ms/step - loss: 0.3412
Epoch 5/5
1/1 [==============================] - 0s 24ms/step


<ipython-input-5-57c49f069867>:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_LSTM_df_1 = result_LSTM_df_1.append(pd.DataFrame({'index': [index],
  0%|          | 2/451 [00:30<1:45:31, 14.10s/it]

...

true_price [[140.158569]
 [140.932266]
 [141.755554]
 [140.991776]
 [141.239761]
 [141.071136]
 [137.242325]
 [139.097198]
 [138.740112]]
stock name AME
train (102, 1)
test (19, 1)
train shape (92, 9, 1)
train label shape (92, 1)
test shape (9, 9, 1)
test lable shape (9, 1)
Epoch 1/5
1/1 [==============================] - 7s 7s/step - loss: 0.5091
Epoch 2/5
1/1 [==============================] - 0s 16ms/step - loss: 0.4685
Epoch 3/5
1/1 [==============================] - 0s 24ms/step - loss: 0.4357
Epoch 4/5
1/1 [==============================] - 0s 18ms/step - loss: 0.4039
Epoch 5/5
1/1 [==============================] - 0s 30ms/step


<ipython-input-5-57c49f069867>:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_LSTM_df_1 = result_LSTM_df_1.append(pd.DataFrame({'index': [index],
100%|█████████▉| 450/451 [1:12:32<00:10, 10.01s/it]

true_price [[75.121048]
 [75.817154]
 [75.150879]
 [74.504478]
 [75.638153]
 [74.673538]
 [73.201744]
 [74.295639]
 [72.624954]]
stock name NTRS
train (102, 1)
test (19, 1)
train shape (92, 9, 1)
train label shape (92, 1)
test shape (9, 9, 1)
test lable shape (9, 1)
Epoch 1/5
1/1 [==============================] - 6s 6s/step - loss: 0.4801
Epoch 2/5
1/1 [==============================] - 0s 25ms/step - loss: 0.4460
Epoch 3/5
1/1 [==============================] - 0s 17ms/step - loss: 0.4134
Epoch 4/5
1/1 [==============================] - 0s 23ms/step - loss: 0.3775
Epoch 5/5
1/1 [==============================] - 0s 19ms/step


<ipython-input-5-57c49f069867>:205: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_LSTM_df_1 = result_LSTM_df_1.append(pd.DataFrame({'index': [index],
100%|██████████| 451/451 [1:12:41<00:00,  9.67s/it]

true_price [[106.165054]
 [106.785385]
 [106.765694]
 [106.194595]
 [105.879509]
 [104.973625]
 [100.729782]
 [102.127983]
 [100.946404]]


In [ ]:
result_LSTM_df_1.head(100)

,index,stock_name,MSE,Daily_Accuracy,Overall_Accuracy,True_Price,Predicted_Price
0,0,COL,[8.497060781166734],"[[0.978803131861785], [0.9753390374246481], [0...",[0.9765022373566669],"[[135.669724], [136.804108], [137.570328], [13...","[132.79395075002398, 132.95648466295734, 133.5..."
1,1,F,[0.07426132188292928],"[[0.9812062461777058], [0.9736130359702556], [...",[0.9746334900157131],"[[10.97576], [11.150132], [11.159821], [11.082...","[10.769484268547416, 10.77124857187228, 10.773..."
2,2,GS,[3.313432779997563],"[[0.9945032634375657], [0.995705142685907], [0...",[0.9951403098186176],"[[234.435989], [235.128403], [235.425171], [23...","[235.72462187228675, 235.85564999311418, 235.9..."
3,3,ETR,[6.520429246947878],"[[0.9877778813718594], [0.9901585150535952], [...",[0.9830111240439492],"[[74.523643], [74.63131], [75.052177], [77.068...","[75.43447980534721, 75.1881231035974, 75.13867..."
4,4,EBAY,[0.032620064818587625],"[[0.9892955792668662], [0.9926986834406234], [...",[0.9943166106450191],"[[38.32], [38.290001], [37.77], [38.040001], [...","[37.90980659750631, 38.14073844385642, 37.9028..."
...,...,...,...,...,...,...,...
95,95,MMC,[0.5222641805459243],"[[0.9955766773405984], [0.9940094160885117], [...",[0.9925489800929012],"[[80.37526700000002], [80.622826], [80.860474]...","[80.01974126022347, 80.01349116444216, 80.0103..."
96,96,KSU,[3.861942712148671],"[[0.9857153707140928], [0.9876622377007056], [...",[0.987045468783357],"[[108.306618], [108.704071], [107.412338], [10...","[106.75949811265963, 107.5745383781767, 107.75..."
97,97,ED,[2.3909604704510605],"[[0.9922968074522969], [0.9931753610940787], [...",[0.9893993808105674],"[[72.640617], [72.512955], [72.807571], [73.75...","[73.20018165953496, 72.94412321318472, 72.9187..."
98,98,MHK,[17.330070328675767],"[[0.9711264249141174], [0.971000161728137], [0...",[0.9764679623937743],"[[216.149994], [216.119995], [215.089996], [21...","[209.9089709184279, 209.82526209856135, 209.80..."


In [ ]:
result_LSTM_df_1.to_csv('/content/drive/MyDrive/Data/result_LSTM_without_News_df.csv')

In [ ]:

# Convert the DataFrame to a NumPy array
data_array = result_LSTM_df_1.to_numpy()

# Save the NumPy array to a binary file using np.save()
np.save('/content/drive/MyDrive/Data/result_LSTM_df.npy', data_array)

In [ ]:
# Load the data back from the binary file
loaded_data = np.load('/content/drive/MyDrive/Data/result_LSTM_df.npy', allow_pickle=True)

# Convert the NumPy array back to a DataFrame
loaded_df = pd.DataFrame(loaded_data, columns=result_LSTM_df_1.columns)

In [ ]:
loaded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             451 non-null    object
 1   stock_name        451 non-null    object
 2   MSE               451 non-null    object
 3   Daily_Accuracy    451 non-null    object
 4   Overall_Accuracy  451 non-null    object
 5   True_Price        451 non-null    object
 6   Predicted_Price   451 non-null    object
dtypes: object(7)
memory usage: 24.8+ KB


## Predicting S&P500


In [3]:
df_price = pd.read_csv('/content/drive/MyDrive/source_price.csv')

cols = ['Adj Close']

# Parameters
split = (0.85);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=1
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0

split_index = int(len(df_price) * split)

data_train = df_price.get(cols).values[ : split_index]
len_train = len(data_train)
print('train', data_train.shape)
data_test = df_price.get(cols).values[ split_index: ]
len_test = len(data_test)
print('test',data_test.shape)


def normalise_data(data_train, sequence_length):  # data_train is numpy array of shape(102,5)
    record_min = []
    record_max = []
    data_windows = []
    for i in range(len(data_train) - sequence_length):
        data_windows.append(data_train[i:i + sequence_length]) # sequence data at every 10 day timestamp
    data_windows = np.array(data_windows).astype(float)
    y_test_ori = data_windows[:, -1, [0]] # only useful for y test

    window_data = data_windows
    win_num = window_data.shape[0]
    col_num = window_data.shape[2]

    normalised_data = []

    for win_i in range(win_num):
        normalised_window = []
        for col_i in range(col_num):
            temp_col = window_data[win_i, :, col_i]
            temp_min = min(temp_col)
            if col_i == 0: # price
                record_min.append(temp_min)
            temp_col = temp_col - temp_min
            temp_max = max(temp_col)
            if col_i == 0:
                record_max.append(temp_max)
            temp_col = temp_col / temp_max
            normalised_window.append(temp_col)
        normalised_window = np.array(normalised_window).T
        normalised_data.append(normalised_window)

    normalised_data = np.array(normalised_data)

    #By using normalised_data[:, :-1], the code is effectively extracting the input data for the LSTM model.
    #  Specifically, it is extracting all features from the first to the penultimate time step for each data sample, resulting in a new array of shape [92, 9, 5].
    #  This is because the input data for each sample has a length of sequence_length - 1 (i.e., 9 in this case) along the second dimension.

    x_train = normalised_data[:, :-1] #---> prices till 9 time stamps

    #y_train = normalised_data[:, -1, [0]]: This line extracts the last column of normalised_data and reshapes it to retain the 2-dimensional structure.
    #  The resulting y_train array contains the target variable that corresponds to the last time step of each input sequence in x_train.
    # In other words, it represents the next data point that the model needs to predict.
    y_train = normalised_data[:, -1, [0]] #---> price to predict after 9th time stamp

    return x_train, y_train, y_test_ori, record_min, record_max, window_data, normalised_data


x_train , y_train , y_train_ori , train_record_min, train_record_max, train_window_data , train_normalised_data = normalise_data(data_train , sequence_length)

x_test , y_test , y_test_ori , test_record_min, test_record_max, test_window_data , test_normalised_data = normalise_data(data_test , sequence_length)






train (102, 1)
test (19, 1)


In [4]:
df_price.head(100)

,Unnamed: 0,date,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,Adj Close
0,0,2017-12-07,-0.880858,-0.496778,0.000000,0.000000,2636.979980
1,1,2017-12-08,0.000000,0.000000,-0.984410,0.000000,2651.500000
2,2,2017-12-11,0.000000,0.000000,0.000000,0.000000,2659.989990
3,3,2017-12-12,0.000000,0.000000,0.000000,0.000000,2664.110107
4,4,2017-12-13,-0.785488,0.000000,0.000000,0.000000,2662.850098
...,...,...,...,...,...,...,...
95,95,2018-04-26,-0.728050,-0.677689,-0.585291,-0.692253,2666.939941
96,96,2018-04-27,-0.739730,-0.638403,-0.562129,-0.674928,2669.909912
97,97,2018-04-30,-0.805050,-0.663419,-0.714391,-0.703474,2648.050049
98,98,2018-05-01,-0.698149,-0.715351,-0.828814,-0.713365,2654.800049


In [5]:
print('train shape',x_train.shape)
print('train label shape',y_train.shape)
#print(y_train_ori)
print('test shape', x_test.shape)
print('test lable shape', y_test.shape)



train shape (92, 9, 1)
train label shape (92, 1)
test shape (9, 9, 1)
test lable shape (9, 1)


In [6]:
# LSTM MODEL parameters
# sequence_length = 10
# batch_size = 100
#input_dim = 5
#input_timesteps = 9
#neurons = 50
#epochs = 5
#prediction_len = 1
#dense_output = 1
#drop_out = 0.2
model = Sequential()
model.add(LSTM(neurons, input_shape = (input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons, return_sequences = True))
model.add(LSTM(neurons, return_sequences = False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))

# Compile Model
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

# Fit Model
model.fit(x_train, y_train, batch_size, epochs )

# Test Data
prediction_sequence = []
window_size = sequence_length
data = x_test
pred_num = int(len(data)/prediction_len)
for i in range(0, pred_num):
  current = data[i * prediction_len]
  predicted = []
  for j in range(0 , prediction_len):
    # By using np.newaxis, we effectively increase the dimensionality of curr_frame from a 2D array
    # (shape: (input_timesteps, input_dim)) to a 3D array (shape: (1, input_timesteps, input_dim)).
    # This is done to match the expected shape for prediction with the LSTM model.
    temp = model.predict(current[newaxis, : , :])[0]
    # the output of model.predict() is a 2D array with shape (1, dense_output)
    #  The [0] indexing is used to extract the predicted value from the 2D array and convert it to a scalar.
    predicted.append(temp)
    # after predicting the value at the current time step,
    # we need to shift the input sequence by one step to include the newly predicted value and exclude the oldest value from the sequence.
    current = current[1:]
    # Sliding window --> [ window_size - 2 ]
    # new_arr = np.insert(arr, index, row_to_insert, axis=0)
    # new_arr = np.insert(arr, index, column_to_insert, axis=1)
    current = np.insert(current , [window_size - 2], predicted[-1], axis = 0)
  prediction_sequence.append(predicted)


# De-Normalise predictions
de_predict = []
len_pre_win = int(len(data)/prediction_len)
len_pre = prediction_len
m = 0
for i in range(0,len_pre_win):
    for j in range(0, len_pre):
        # prediction_seqs[i][j][0] retrieves the normalized prediction for the j-th prediction in the i-th window.
        # This prediction is denormalized back to the original scale using record_max[m] and record_min[m].
        # m is used to keep track of the corresponding maximum and minimum values used for normalization during the data preparation step.
        de_predict.append(prediction_sequence[i][j][0] * test_record_max[m] + test_record_min[m])
        # m is incremented after each prediction to keep track of the current index in record_max and record_min.
        m += 1
#print('denormalized predictions', de_predict)


# Error
error = []
diff = y_test.shape[0] - prediction_len * len_pre_win
for i in range(y_test_ori.shape[0] - diff):
    error.append(y_test_ori[i,] - de_predict[i])

squared_error = []
absolute_error = []
for value in error:
    squared_error.append(value*value)
    absolute_error.append(abs(value))

error_percent = []
for i in range(len(error)):
    val = absolute_error[i]/y_test_ori[i,]
    val = abs(val)
    error_percent.append(val)

mean_error_percent = sum(error_percent)/len(error_percent)
accuracy = 1 - mean_error_percent

MSE = sum(squared_error)/len(squared_error)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)


Epoch 1/5
1/1 [==============================] - 10s 10s/step - loss: 0.4873
Epoch 2/5
1/1 [==============================] - 0s 15ms/step - loss: 0.4556
Epoch 3/5
1/1 [==============================] - 0s 14ms/step - loss: 0.4255
Epoch 4/5
1/1 [==============================] - 0s 15ms/step - loss: 0.3962
Epoch 5/5
1/1 [==============================] - 0s 18ms/step
MSE [580.14692859]
accuracy [0.99264079]
mean_error_percent [0.00735921]


In [7]:


# Save the NumPy array to a binary file using np.save()
np.save('/content/drive/MyDrive/sp500_1dim_pred.npy', de_predict)
np.save('/content/drive/MyDrive/sp500_1dim_true.npy', y_test_ori)

In [8]:
print(de_predict)

[2681.5161105703296, 2682.9830544935444, 2703.8784071063924, 2714.8753337295702, 2714.8965054025825, 2714.847612522788, 2698.405396547044, 2698.6694486613023, 2698.5808820698035]


In [9]:
print(y_test_ori)

[[2712.969971]
 [2733.01001 ]
 [2724.439941]
 [2733.290039]
 [2727.76001 ]
 [2721.330078]
 [2689.860107]
 [2724.01001 ]
 [2705.27002 ]]
